In [1]:
import numpy as np;
import pandas as pd;
from scipy.integrate import odeint


In [2]:
# Global parameters
NO_OF_DAYS = 100
beta = 0.49  
gamma = 0.11
delta = 0.2
mu = 0.3

# System variables
ROOT_DIR = "/Users/sreeramh/Documents/Kerala_Covid/SpatioTemporal/"

In [3]:
# Load data
initDataDF = pd.read_csv(ROOT_DIR + "data/sample_init_data.csv")
print(initDataDF)

neighborDF = pd.read_csv(ROOT_DIR + "data/sample_neighbor_data.csv")
print(neighborDF)


  name      S  E  I  R
0   P1  14473  0  0  0
1   P2  19804  0  0  0
2   P3   1996  0  2  0
3   P4  16648  0  0  0
4   P5   2456  0  0  0
5   P6  16205  0  1  0
6   P7  11931  0  0  0
7   P8   6934  0  3  0
8   P9   8587  0  2  0
9  P10   7944  0  0  0
  fromName toName  distance
0       P3     P1      0.03
1       P3     P2      0.01
2       P1     P3      0.03
3       P2     P3      0.01


In [4]:
## Differential equation
def deriv(y, N, beta, gamma, delta, contactRatio):
    S, E, I, R = y
    ## TODO what to do with contactRatio;
    dSdt = -beta * S * I / N
    dEdt = beta * S * I / N - delta * E
    dIdt = delta * E - gamma * I
    dRdt = gamma * I
    return dSdt, dEdt, dIdt, dRdt

In [5]:
def findSpatialContactRatio(name, NDict, yDict, neighborDF):
    ## TODO how to compute this
    contactRatio = 0;
    neighbors = neighborDF[neighborDF['fromName']==name]
    for index, row in neighbors.iterrows():
        #print(name, row['toName'], row['distance'], NDict[name], NDict[row['toName']])
        contactRatio = contactRatio + (mu * (1/row['distance']) * ((NDict[name] * NDict[row['toName']])/(NDict[name] + NDict[row['toName']])))
    return contactRatio;
        

In [6]:
## next count 
def derivNext(y, rates) :
    S,E,I,R = y
    dSdt, dEdt, dIdt, dRdt = rates
    S1 = S + dSdt
    E1 = E + dEdt
    I1 = I + dIdt
    R1 = R + dRdt
    return S1, E1, I1, R1


In [7]:
yDict = {};
NDict = {};
resultsDF = pd.DataFrame(columns=['name', 'day', 'S', 'E', 'I', 'R'], );

for index, row in initDataDF.iterrows():
    NDict[row['name']] = row['S'] + row['E'] + row['I'] + row['R']
    yDict[row['name']] = row['S'], row['E'], row['I'], row['R']

for day in range(1,NO_OF_DAYS):
    for index, row in initDataDF.iterrows():
        name = row['name'];
        y = yDict[name]; 
        
        contactRatio = findSpatialContactRatio(name, NDict, yDict, neighborDF);
        #print(name, contactRatio);
        
        rates = deriv(y, NDict[name], beta, gamma, delta, contactRatio)
        yNext = derivNext(y, rates) 
        
        yDict[name] = yNext
        #print(name, yNext)
        resultsDF = resultsDF.append({'name': name, 'day': day, 'S': yNext[0], 'E': yNext[1], 'I': yNext[2], 'R': yNext[3]},ignore_index=True)


In [8]:
resultsDF

,name,day,S,E,I,R
0,P1,1,14473.000000,0.000000,0.000000,0.000000
1,P2,1,19804.000000,0.000000,0.000000,0.000000
2,P3,1,1995.020981,0.979019,1.780000,0.220000
3,P4,1,16648.000000,0.000000,0.000000,0.000000
4,P5,1,2456.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...
985,P6,99,196.109563,56.555130,690.868651,15262.466656
986,P7,99,11931.000000,0.000000,0.000000,0.000000
987,P8,99,71.433901,4.557642,79.001562,6782.006896
988,P9,99,91.205904,9.368120,149.914233,8338.511743


In [9]:
resultsDF.to_csv(ROOT_DIR + "output/results.csv")